In [1]:
# example_meta_data_
# reader.ipynb
# Authors: Stephan Meighen-Berger
# Shows how to deal with the output files

In [2]:
import awkward as ak
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import json
%matplotlib widget
# %matplotlib qt

In [3]:
# Example loading a directory (highly recommend using lazy loading)
example_set = ak.from_parquet('./output/1337_photons.parquet')

In [5]:
meta = pq.read_metadata('./output/1337_photons.parquet').metadata[b'config_prometheus']
jmeta = json.loads(meta)
jmeta['general']

{'version': 'github'}

In [6]:
jmeta['injection']

{'name': 'LeptonInjector',
 'LeptonInjector': {'inject': True,
  'paths': {'install location': '/opt/LI/install/lib/python3.9/site-packages',
   'xsec dir': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/resources/cross_section_splines/',
   'earth model location': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/resources/earthparams/densities/PREM_water.dat',
   'injection file': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/examples/output/1337_LI_output.h5',
   'lic file': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/examples/output/1337_LI_config.lic',
   'diff xsec': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/resources/cross_section_splines/dsdxdy_nubar_CC_iso.fits',
   'total xsec': '/home/unimelb.edu.au/smeighenberg/Projects/prometheus/resources/cross_section_splines/sigma_nubar_CC_iso.fits'},
  'simulation': {'final state 1': 'MuMinus',
   'final state 2': 'Hadrons',
   'minimal energy': 10000.0,
   'maximal energy': 100000.0,
   'p

In [7]:
# Example loading a single file
# example_set = ak.from_parquet('./output/custom_1337_meta_data.parquet')

In [8]:
# The events are split into three parts:
#   - lepton: the light from the generated lepton
#   - hadron: the light produced from the associated hadronic cascade (at production point)
#   - total: the light from both parts
# Each of these fields is formatted the same way
example_set.fields

['mc_truth', 'photons']

In [9]:
# Example for one event
event_id = 0
# The sensors hit
print("Total number of hits:")
print(len(example_set.photons.sensor_id[event_id]))
print('The hit sensors: ')
print(example_set.photons.sensor_id[event_id])
print('Their associated strings')
print(example_set.photons.string_id[event_id])
print('The hit times')
print(example_set.photons.t[event_id])

Total number of hits:
1161
The hit sensors: 
[44, 48, 42, 43, 43, 44, 44, 44, 44, ..., 46, 46, 46, 46, 46, 47, 47, 47, 47]
Their associated strings
[7, 7, 15, 16, 16, 16, 16, 16, 16, 16, ..., 26, 26, 26, 26, 26, 26, 26, 26, 26]
The hit times
[1.16e+03, 1.53e+03, 1.49e+03, 1.09e+03, 997, ..., 1.17e+03, 809, 809, 928, 789]


In [10]:
example_set.photons.sensor_pos_x

<Array [[-540, -540, -405, ..., -270, -270], ...] type='10 * var * float64'>

In [11]:
def parquet_sub_plotting(
        data, e_id, det,
        channel='photons',
        scaling=20,
        number_of_hits=1000,
        hitcut=3,
        offset=0
    ):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    hit_counts = []
    print(len(data[channel].sensor_pos_x[e_id])-1)
    subset_ids = np.random.randint(
        0,
        len(data[channel].sensor_pos_x[e_id])-1,
        size=number_of_hits
    )
    print('Total amount of hits: %d' %len(data[channel].sensor_pos_x[e_id]))
    print('Using a subset of %d for plotting' %number_of_hits)
    event_id = e_id
    sensor_comb = np.array([
        [data[channel].sensor_pos_x[event_id][subset_ids][i],
        data[channel].sensor_pos_y[event_id][subset_ids][i],
        data[channel].sensor_pos_z[event_id][subset_ids][i]
        ]
        for i in range(number_of_hits)
    ])
    print('Fetching unique modules and their hits')
    hit_doms, hit_counts = np.unique(sensor_comb, axis=0, return_counts=True)
    print('Number of hit doms: %d' %len(hit_doms))
    hit_counts = np.log10(hit_counts)
    hit_counts[hit_counts < np.log10(hitcut)] = 0.
    print('Finished the modules')
    print('Plotting...')
    ax.scatter(
        hit_doms[:, 0],
        hit_doms[:, 1],
        hit_doms[:, 2]+offset,
        s=(
            hit_counts * scaling
        ),
        color='r', alpha=0.5)
    print('Finished modules')
    ax.scatter(
        det[:, 0],
        det[:, 1],
        det[:, 2]+offset,
        s=1,
        color='k', alpha=0.5)
    # ax.set_xlim(min(det[:, 0]), max(det[:, 0]))
    # ax.set_ylim(min(det[:, 1]), max(det[:, 1]))
    # ax.set_zlim(min(det[:, 2]), max(det[:, 2]))
    ax.set_xlabel('X [m]')
    ax.set_ylabel('Y [m]')
    ax.set_zlabel('Z [m]')
    plt.title(
        'Event %d with pdg id = %d and energy %.1f GeV' % (
            event_id,
            data.mc_truth.final_state_type[event_id][0],
            data.mc_truth.final_state_energy[event_id][0]
            )
    )
    plt.show()

In [12]:
# Getting the detector
pos = []
with open('../resources/geofiles/demo_water.geo') as geo_in:
    read_lines = geo_in.readlines()
    modules_i = read_lines.index("### Modules ###\n")   
    for line in read_lines[modules_i+1:]:
        line = line.strip("\n").split("\t")
        pos.append(
            np.array([float(line[0]), float(line[1]),
            float(line[2])]))
    det = np.array(pos)
det = det#  - np.mean(det, axis=0)

In [15]:
parquet_sub_plotting(
    example_set,
    e_id=6, det=det,
    channel='photons',
    scaling=10,
    number_of_hits=1000,
    hitcut=0,
    offset=3500
)

1099
Total amount of hits: 1100
Using a subset of 1000 for plotting
Fetching unique modules and their hits
Number of hit doms: 59
Finished the modules
Plotting...
Finished modules


/tmp/ipykernel_52217/127794781.py:32: RuntimeWarning: divide by zero encountered in log10
  hit_counts[hit_counts < np.log10(hitcut)] = 0.
